# Programación Dinámica
Francisco Castillo - 21562
Diego Lemus -

## Task 1
### ¿Qué es la Programación Dinámica y cómo se relaciona con Reinforcement Learning?
Es un paradigma de programación que se utiliza para resolver problemas que pueden descomponerse en otros subproblemas más simples. En los Markov Decision Processes, la programación dinámica permite encontrar la política óptima y el valor de los estados mediante la iteración de políticas o la iteración de valores mientras se conozcan las dinámicas del entorno (probabilidades de transición y recompensas).
### Explique en sus propias palabras el algoritmo de Iteración de Póliza
El algoritmo de Policy Iteration es un método para encontrar la política óptima en un MDP. Consiste en dos pasos:
1. **Evaluación de la política:** se calcula el valor de los estados bajo la política actual.
2. **Mejora de la política:** se actualiza la política seleccionando acciones que maximicen el valor esperado de los estados. Este proceso se repite hasta que la política converge.
### Explique en sus propias palabras el algoritmo de Iteración de Valor
El algoritmo de Value Iteration es un método para encontrar la política óptima en un MDP mediante la actualización iterativa de los valores de los estados. Este solo tiene un paso:
1. **Actualización de valores:** se actualizan los valores de los estados utilizando la ecuación de Bellman, considerando las acciones posibles y las probabilidades de transición. Este proceso se repite hasta que los valores convergen, lo que permite derivar la política óptima a partir de los valores finales.
### En el laboratorio pasado, vimos que el valor de los premios obtenidos se mantienen constantes, ¿por qué?
Se mantienen constantes porque era un ambiente determinista con recomepnsas fijas. Esto implica que la función de recomensa no cambia con el tiempo y las recompensas son predecibles.

> [Value Iteration vs. Policy Iteration](https://www.geeksforgeeks.org/data-science/what-is-the-difference-between-value-iteration-and-policy-iteration/)